# Image-to-Image Search with ArcFaceLoss

<a href="https://colab.research.google.com/drive/1ZS9FmnR9FzO_JYGCPazFM7TcMNQl51xM?usp=sharing"><img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg"></a>

Searching visually similar images with image queries is a very popular use case. However, using pre-trained models does not deliver the best results – the models are trained on general data that lack the particularities of your specific task. Here's where Finetuner comes in! It enables you to accomplish this easily.

This guide will demonstrate how to fine-tune a ResNet model using the `ArcFaceLoss` function.

*Note, please consider switching to GPU/TPU Runtime for faster inference.*

## Install

In [ ]:
!pip install 'finetuner[full]'

## Task

More specifically, we will fine-tune ResNet50 on [Stanford Cars Dataset](http://ai.stanford.edu/~jkrause/cars/car_dataset.html).
This dataset consists of 196 classes across 16184 docuemnts in total.

Each class of the dataset consists of roughly 82 documents, in order to group the documents of each class together and move separate documents of different classes apart, we use the `ArcFaceLoss` function. For more information on how this loss function works, see [Advanced Losses and Optimizers](https://finetuner.jina.ai/advanced-topics/advanced-losses-and-optimizers/)

After fine-tuning, documents from each class should have similar embeddings, distinct from documents of other classes.

## Data

Our journey starts locally. We have to prepare the data and push it to the Jina AI Cloud and Finetuner will be able to get the dataset by its name. For this example,
we already prepared the data, and we'll provide the names of training data (`stanford-cars-train`) directly to Finetuner.

```{important} 
We don't require you to push data to the Jina AI Cloud by yourself. Instead of a name, you can provide a `DocumentArray` and Finetuner will do the job for you.
When working with documents where images are stored locally, please call `doc.load_uri_to_blob()` to reduce network transmission and speed up training.
```

In [ ]:
import finetuner
from docarray import DocumentArray, Document

finetuner.login(force=True)

In [ ]:
train_data = DocumentArray.pull('finetuner/stanford-cars-train', show_progress=True)
query_data = DocumentArray.pull('finetuner/stanford-cars-query', show_progress=True)
index_data = DocumentArray.pull('finetuner/stanford-cars-index', show_progress=True)

train_data.summary()

## Backbone model
Now let's see which backbone models we can use. You can see available models by calling `finetuner.describe_models()`.


For this example, we're gonna go with `resnet50`.

## Fine-tuning

Now that we have the training and evaluation datasets loaded as `DocumentArray`s and selected our model, we can start our fine-tuning run.

In [ ]:
from finetuner.callback import EvaluationCallback

run = finetuner.fit(
    model='resnet50',
    train_data='finetuner/stanford-cars-train',
    batch_size=128,
    epochs=5,
    learning_rate=1e-3,
    loss='ArcFaceLoss',
    device='cuda',
    sampler='random',
    callbacks=[
        EvaluationCallback(
            query_data='finetuner/stanford-cars-query',
            index_data='finetuner/stanford-cars-index',
        )
    ],
)

Let's understand what this piece of code does:

* As you can see, we have to provide the `model` which we picked before.
* We also set `run_name` and `description`, which are optional,
but recommended in order to retrieve your run easily and have some context about it.
* Furthermore, we had to provide names of the `train_data`.
* We set `ArcFaceLoss`.
* Additionally, we use `finetuner.callback.EvaluationCallback` for evaluation.
* Lastly, we set the number of `epochs` and provide a `learning_rate`.

## Monitoring

Now that we've created a run, let's see its status. You can monitor the run by checking the status - `run.status()` - and the logs - `run.logs()` or `run.stream_logs()`. 

In [ ]:
# note, the fine-tuning might takes 30~ minutes
for entry in run.stream_logs():
    print(entry)

Since some runs might take up to several hours, it's important to know how to reconnect to Finetuner and retrieve your runs.

```python
import finetuner
finetuner.login()

run = finetuner.get_run(run.name)
```

You can continue monitoring the runs by checking the status - `finetuner.run.Run.status()` or the logs - `finetuner.run.Run.logs()`. 

## Evaluating
Currently, we don't have a user-friendly way to get evaluation metrics from the `finetuner.callback.EvaluationCallback` we initialized previously.
What you can do for now is to call `run.logs()` after the end of the run and see the evaluation results:

```bash
Training [5/5] ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48/48 0:00:00 0:00:12 • loss: 13.986
INFO     Done ✨                                                                              __main__.py:195
DEBUG    Finetuning took 0 days, 0 hours 3 minutes and 48 seconds                             __main__.py:197
INFO     Metric: 'resnet50_precision_at_k' before fine-tuning:  0.11575 after fine-tuning:    __main__.py:210
0.53425
INFO     Metric: 'resnet50_recall_at_k' before fine-tuning:  0.05745 after fine-tuning:       __main__.py:210
0.27113
INFO     Metric: 'resnet50_f1_score_at_k' before fine-tuning:  0.07631 after fine-tuning:     __main__.py:210
0.35788
INFO     Metric: 'resnet50_hit_at_k' before fine-tuning:  0.82900 after fine-tuning: 0.94100  __main__.py:210
INFO     Metric: 'resnet50_average_precision' before fine-tuning:  0.52305 after fine-tuning: __main__.py:210
0.79779
INFO     Metric: 'resnet50_reciprocal_rank' before fine-tuning:  0.64909 after fine-tuning:   __main__.py:210
0.89224
INFO     Metric: 'resnet50_dcg_at_k' before fine-tuning:  1.30710 after fine-tuning: 4.52143  __main__.py:210
INFO     Building the artifact ...                                                            __main__.py:215
INFO     Pushing artifact to Jina AI Cloud ...                                                __main__.py:241
[12:19:53] INFO     Artifact pushed under ID '63f8a9089c6406e19244771d'                                  __main__.py:243
DEBUG    Artifact size is 83.580 MB                                                           __main__.py:245
INFO     Finished 🚀                                                                          __main__.py:246
```

## Saving

After the run has finished successfully, you can download the tuned model on your local machine:


In [ ]:
artifact = run.save_artifact('resnet-model')

## Inference

Now you saved the `artifact` into your host machine,
let's use the fine-tuned model to encode a new `Document`:

```{admonition} Inference with ONNX
In case you set `to_onnx=True` when calling `finetuner.fit` function,
please use `model = finetuner.get_model(artifact, is_onnx=True)`
```

In [ ]:
query = DocumentArray([query_data[0]])

model = finetuner.get_model(artifact=artifact, device='cuda')

finetuner.encode(model=model, data=query)
finetuner.encode(model=model, data=index_data)

assert query.embeddings.shape == (1, 2048)

And finally, you can use the embedded `query` to find top-k visually related images within `index_data` as follows:

In [ ]:
query.match(index_data, limit=10, metric='cosine')

## Before and after
We can directly compare the results of our fine-tuned model with its zero-shot counterpart to get a better idea of how finetuning affects the results of a search. Each class of the Stanford cars dataset contains images for a single model of car. Therefore, we would expect the search results after finetuning to return more images of cars that are of the same model as the car in the query image. The improvement in results can be quite subtle for some queries, though the example below clearly shows that fine-tuning increases the quality of the search results

query                      |before             |  after
:-------------------------:|:-------------------------:|:-------------------------:
![cars-query](https://user-images.githubusercontent.com/58855099/221186269-a7ebbcd0-6865-45ea-b539-9756d87b3853.png) | ![cars-result-zs](https://user-images.githubusercontent.com/58855099/221186221-6d5bfb9b-2a44-4436-a1af-4c6763eb3b5b.png)  |  ![cars-result-ft](https://user-images.githubusercontent.com/58855099/221187091-adf30d01-9773-4fa6-8e32-b2f45916ff55.png)
